# 100x10 Hearthstone Arena Challenge
### The rules:
In 10 arena runs in a row get 100 wins. You cannot pick a hero you played before unless all three choices are heroes you've played before.

The popular Hearthstone streamer, Hafu, is the closest person to completing this challenge at 96 wins. I'll use statistics from her stream for my simulation.

In [137]:
#class name, mean, standard deviation
probs <- array(c("Paladin",8.33,2.5,"Rogue",8.15,3.22,"Mage",7.86,2.59,
                 "Druid",7.84,3.02,"Priest",7.26,2.65,"Warlock",6.82,3.15,
                 "Warrior",6.77,2.83,"Hunter",6.74,2.81,"Shaman",6.64,2.61),c(3,9))

# stats from here https://docs.google.com/spreadsheets/d/1jck4WkEvaKWE0Iw9RrttLHoFO3KEMG6In6XS1okzmx4/edit#gid=1077822399

I'm going to assume that the distribution is normal. It very likely could not be normal, but I don't konw.

In [138]:
#The code to generate the 10 games for one attempt at the challenge
sim <- function(fun){
  classes <- 1:9
  playedClasses <- NULL
  run <- array(0,c(2,11)) # an array to put the results of the run into
  choices <- sample(9,3) # gets 3 classes to choose from
  chosen <- fun(choices, run) # selects the 1 class from the 3 chosen
  playedClasses[1] <- chosen # adds this class to a list that keeps track of the classes played
  run[1,1] = probs[1,chosen] # puts the class name into the run array
  # A random number generated from a normal distribution using the classes mean and standard diviation.
  wins = floor(rnorm(1,mean = as.double(probs[2,chosen]),sd = as.double(probs[3,chosen])))
  if(wins > 12) wins = 12 # lowers the number to 12 if it goes above it
  run[2,1] <- wins # puts the number of wins in the results array
  for(i in 2:10){ # the second to tenth runs
    choices <- sample(9,3) # gets 3 classes to choose from
    # gets a list of classes that are from the 3 to choose from and classes already played
    temp <- intersect(choices, playedClasses) 
    # if all three arent the same then it uses the ones from the original list that havent been played
    if(length(temp) != 3) choices = setdiff(choices,temp) 
    # chooses 1 class from the ones that are allowed
    chosen <- fun(choices, run)
    run[1,i] = probs[1,chosen]
    wins <- floor(rnorm(1,mean = as.double(probs[2,chosen]),sd = as.double(probs[3,chosen])))
    if(wins > 12) wins = 12
    run[2,i] <- wins
    playedClasses[i] <- chosen
  }
  # hack fix because streamer will only choose to start run on 12 wins.
  run[2,1] = "12"
  # add total wins to results array
  run[1,11] <- "total"
  run[2,11] <- sum(as.double(run[2,]))
  run
}

In [139]:
# strategy functions i'll use to determine the selection

selectBadFirst <- function(choices, run){
    if(length(choices)==1){
        return(choices[1])
    }

    #if first game, pick worst class
    if(run[2,1] == 0){
        return(max(choices))
    }else {
        return(min(choices))
    }
}

selectBestAlways <- function(choices, run){

    return(min(choices))
}

In [140]:
# find first n successful runs with selecting the best class always
classlevels.allRuns <- 0
totalAttempts <- 0
n = 5000
for(j in 1:n){
    win = TRUE
    run <- NULL
    count = 0
    
    while(win){
      run <- sim(selectBestAlways)
      count = count + 1
      if(as.numeric(run[2,11]) >= 100){
        classLevelsTotal <- NULL
        classLevels <- NULL
        for(i in 1:10){
            if(i >1)
            classLevelsTotal[i] <- classLevelsTotal[i-1] + (which(run[1,i]==probs[1,]))
            else
            classLevelsTotal[i] <- which(run[1,i]==probs[1,])
            classLevels[i] <- which(run[1,i]==probs[1,])
        }
        totalAttempts <- c(totalAttempts,count)
        classlevels.allRuns <- c(classlevels.allRuns,classLevelsTotal[10])
        win = FALSE
      }
    }

}


In [141]:
c(sum(classlevels.allRuns),mean(classlevels.allRuns),sd(classlevels.allRuns))
c(sum(totalAttempts), mean(totalAttempts), sd(totalAttempts))

[1] 206666.00000     41.32494      4.55045

[1] 6477581.000    1295.257    1318.187

In [142]:
# find first n successful runs with selecting a bad class first
classlevels.allRuns <- 0
totalAttempts <- 0
for(j in 1:n){
    win = TRUE
    run <- NULL
    count = 0
    
    while(win){
      run <- sim(selectBadFirst)
      count = count + 1
      if(as.numeric(run[2,11]) >= 100){
        classLevelsTotal <- NULL
        classLevels <- NULL
        for(i in 1:10){
            if(i >1)
            classLevelsTotal[i] <- classLevelsTotal[i-1] + (which(run[1,i]==probs[1,]))
            else
            classLevelsTotal[i] <- which(run[1,i]==probs[1,])
            classLevels[i] <- which(run[1,i]==probs[1,])
        }
        totalAttempts <- c(totalAttempts,count)
        classlevels.allRuns <- c(classlevels.allRuns,classLevelsTotal[10])
        win = FALSE
      }
    }

}


In [143]:
c(sum(classlevels.allRuns),mean(classlevels.allRuns),sd(classlevels.allRuns))
c(sum(totalAttempts), mean(totalAttempts), sd(totalAttempts))

[1] 2.102970e+05 4.205099e+01 4.450843e+00

[1] 4337646.0000     867.3557     869.7962